#   Uploading hardhat dataset using kaggle in colab

In [ ]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tanugoyal10","key":"c55d825391c7e0c5ba7076994b581bc1"}'}

In [2]:
!mkdir -p ~/.kaggle

In [3]:
!mv kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d muhammetzahitaydn/hardhat-vest-dataset-v3 -p /content/

Dataset URL: https://www.kaggle.com/datasets/muhammetzahitaydn/hardhat-vest-dataset-v3
License(s): CC0-1.0
100% 4.19G/4.21G [00:35<00:00, 128MB/s]
100% 4.21G/4.21G [00:35<00:00, 127MB/s]


In [ ]:
import zipfile
import os
import yaml
import cv2
from ultralytics import YOLO


In [ ]:
#unziping the file

dataset_path = "/content/hardhat-vest-dataset-v3.zip"  
extract_path = "/content/dataset/"

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")

Dataset extracted successfully!


# Checking the structure of data

In [ ]:


# print directory structure and file count
def print_dataset_structure(root_dir):
    for root, dirs, files in os.walk(root_dir):
        indent_level = root.replace(root_dir, "").count(os.sep)
        indent = " " * 4 * indent_level
        num_files = len(files)
        print(f"{indent}- {os.path.basename(root)} - {num_files} files")


print_dataset_structure("/content/dataset")

- dataset - 0 files
    - images - 0 files
        - val - 2438 files
        - test - 2455 files
        - train - 17248 files
    - labels - 1 files
        - val - 2438 files
        - test - 2455 files
        - train - 17248 files


In [ ]:


dataset_path = "/content/dataset"
train_path = "images/train"
val_path = "images/val"
test_path = "images/test"

class_names = ["helmet", "vest", "head"]
data = {
    "path": dataset_path,
    "train": train_path,
    "val": val_path,
    "test": test_path,
    "nc": len(class_names),
    "names": class_names
}

# Save to YAML file
yaml_file = "dataset/data.yaml"
with open(yaml_file, "w") as f:
    yaml.dump(data, f, default_flow_style=False)

print(f"✅ data.yaml created successfully at: {yaml_file}")

✅ data.yaml created successfully at: dataset/data.yaml


In [ ]:
# checking annotation are in correct format or not
def check_annotation_format(label_dir):
    for root, _, files in os.walk(label_dir):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    for line in f.readlines():
                        parts = line.strip().split()

                        if len(parts) != 5:
                            print(f"❌ Incorrect format in {file_path}: {line}")
                            continue



In [10]:
check_annotation_format("dataset/labels")

❌ Incorrect format in dataset/labels/classes.txt: helmet

❌ Incorrect format in dataset/labels/classes.txt: vest

❌ Incorrect format in dataset/labels/classes.txt: head

❌ Incorrect format in dataset/labels/classes.txt: person



In [ ]:
#checking images are in correct format or not

def check_images(directory):
    valid_extensions = {".jpg", ".jpeg", ".png"}
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            ext = os.path.splitext(file)[1].lower()

            if ext in valid_extensions:
                img = cv2.imread(file_path)
                if img is None:
                    print(f"❌ Corrupted image: {file_path}")

In [12]:
check_images("dataset/images")

# Training the Yolov8 algorithm on dataset

In [ ]:
# loading and training yolo model
model = YOLO("yolov8n.pt")  
model.train(data="/content/dataset/data.yaml", epochs=30, imgsz=640, batch=16, device="cuda")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 268MB/s]


Ultralytics 8.3.81 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

100%|██████████| 755k/755k [00:00<00:00, 97.2MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      30/30      5.61G       1.37     0.6495      1.096         60        640: 100%|██████████| 1078/1078 [05:16<00:00,  3.41it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:27<00:00,  2.83it/s]


                   all       2438      21032      0.868      0.823      0.875       0.54

30 epochs completed in 3.008 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.81 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:31<00:00,  2.44it/s]


                   all       2438      21032      0.862      0.826      0.876       0.54
                helmet       1782       6586      0.921      0.861      0.915      0.606
                  vest        406        870      0.785      0.744      0.821      0.538
                  head        712      13576      0.881      0.873      0.891      0.476
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x791acea8a490>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

# Testing the dataset

In [15]:
metrics = model.val(data="/content/dataset/data.yaml", split="test")

Ultralytics 8.3.81 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/dataset/labels/test... 2455 images, 2 backgrounds, 0 corrupt: 100%|██████████| 2455/2455 [00:09<00:00, 250.60it/s]


val: New cache created: /content/dataset/labels/test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 154/154 [00:32<00:00,  4.70it/s]


                   all       2455      20193      0.877      0.832      0.887      0.545
                helmet       1822       6749      0.927      0.849      0.911      0.608
                  vest        440        935      0.813      0.769      0.853      0.547
                  head        713      12509      0.892      0.877      0.896       0.48
Speed: 0.7ms preprocess, 4.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train2
